#<font color='#7800FF'>**Navigating College Scorecard Data with Jupyter Notebooks**</font>
<font color='#64CCC9'>We will work through accessing and examining College Scorecard data via an API. This is not meant to be a rigorous dive into any of these tools. This is an introduction to what is possible through (slightly) advanced data analytics</font>


#<font color='#7800FF'>**This notebook leverages a color code:**</font>

*   <font color='#2961FF'>Section Title</font>
*   <font color='#E04F39'>A Code cell for you to run</font>
*   <font color='#FFCD00'>Details deemed important</font>
*   <font color='#64CCC9'>Details you can skip for now as we will speak to them together</font>



In [ ]:
# @title <font color='#E04F39'>Let's test whether you're ready to proceed</font>
print('Hello World!')

## <font color='#2961FF'>**Stage 0: Intro to our tools**</font>
<font color='#64CCC9'>This section shares references to better understand the tools we will use</font>


### <font color='#64CCC9'>APIs</font>

![image.png](https://raw.githubusercontent.com/dlyczak3/SAIR_2023/main/What-is-an-API-definition.webp)
###### <font color='#64CCC9'>image source: https://rapidapi.com/blog/wp-content/uploads/2023/03/What-is-an-API-definition.webp<font/>

### <font color='#64CCC9'>College Scorecard</font>

<font color='#64CCC9'>Compendium of institute level, time series data from the US Department of Education</font>

<font color='#64CCC9'>Established to increase transparency and enable fair comparisons between colleges</font>

<font color='#FFCD00'>If you already have a scorecard API key, locate your key, and have it ready for use later in this workbook</font>

<font color='#FFCD00'>If you are unable to request an API key (which should not be the case), you will use a sample of data from a CSV</font>

##### <font color='#64CCC9'>Reference to College Scorecard API: https://github.com/RTICWDT/open-data-maker/blob/master/API.md</font>

### <font color='#FFCD00'>Gain access to ScoreCard API</font>
1.   Go to the Key Request page: https://collegescorecard.ed.gov/data/documentation/
2.   Scroll down to this section:
![image.png](https://raw.githubusercontent.com/dlyczak3/SAIR_2023/main/scorecard_api.png)

3. If you're not a robot / bot, please click and complete the Captcha

4. Complete the 3 required fields



### <font color='#64CCC9'>Python</font>

<font color='#64CCC9'>Python is a high level programming language</font>

<font color='#64CCC9'>Further details on Python (and our primary library Pandas) are beyond the scope of this demo</font>

<font color='#64CCC9'>However, to help (and show video capabilities of Jupyter) run the cell below by clicking the $\blacktriangleright$ button</font>

<font color='#64CCC9'>Note, the $\blacktriangleright$ icon is a result of Latex. If you're familiar with Latex, its functionality can be used natively in Jupyter</font>

In [ ]:
#@title <font color='#E04F39'>Running this code shows how you can add video</font>
from IPython.display import Audio, Image, YouTubeVideo
id = 'kqtD5dpn9C8?si=kgYhoEKm5uqmsjwy'
YouTubeVideo(id=id,width = 600, height = 300)

## <font color='#2961FF'>**Stage 1: Loading Data**</font>

<font color='#64CCC9'>To further demo Jupyter Notebooks, we will walk through a quick analysis using scorecard data</font>

<font color='#64CCC9'>Interactivity was built into the notebook. For a handy reference on building your own: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html</font>

In [ ]:
#@title <font color='#E04F39'>Run this cell to check your api key or to opt out of using it</font>
try:
    from ipywidgets import interact, interactive, fixed, interact_manual, Layout, Button, HBox, VBox
    import ipywidgets as widgets
except:
    import micropip
    await micropip.install('ipywidgets')
    from ipywidgets import interact, interactive, fixed, interact_manual, Layout, Button, HBox, VBox
    import ipywidgets as widgets
import requests
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def user_input(response):
    return response

def test_key(key):
    key = key.strip()
    try:
        r = requests.get(f"https://api.data.gov/ed/collegescorecard/v1/schools.json?latest.school.name=Georgia%20Institute%20of%20Technology&fields=latest.school.name&api_key={key}")
        data = r.json()
        if data['results'][0]['latest.school.name'] == "Georgia Institute of Technology-Main Campus":
            print('your API key was successfully entered, please continue')
            return key
        else:
            print('There was an error with your API Key. Please try again or choose \'N\' for accessing via API')
            return 'N'
    except:
        print('There was an error with your API Key. Please try again or choose \'N\' for accessing via API')
        return 'N'

has_key = input("Would you like to access the API? Y/N: ")
if has_key.strip()[0].lower() == 'y':
    api_key = input("Please copy and paste your key here: ")
    api_key = test_key(api_key)
else:
    api_key = 'N'
    print('You will be using a pre-processed subsection of scorecard data')


## <font color='#FFCD00'>***NOTE If you do not provide a valid key, the next few sections do not influence your data***</font>

In [ ]:
#@title <font color='#E04F39'>Select the states with universities you're interested in</font>
GA = interactive(user_input, response = widgets.Checkbox(value = False, description = 'Georgia Schools', disabled = False, indent = False,));
TN = interactive(user_input, response = widgets.Checkbox(value = False, description = 'Tennessee Schools', disabled = False, indent = False,));
TX = interactive(user_input, response = widgets.Checkbox(value = False, description = 'Texas Schools', disabled = False, indent = False,));
NC = interactive(user_input, response = widgets.Checkbox(value = False, description = 'North Carolina Schools', disabled=False, indent = False,));
FL = interactive(user_input, response = widgets.Checkbox(value = False, description = 'Florida Schools', disabled = False, indent = False,));
AL = interactive(user_input, response = widgets.Checkbox(value = False, description = 'Alabama Schools', disabled = False, indent = False,));

print('You\'ll be given a list of schools (10 from each state) please choose the states you want to see schools for')
display(widgets.HBox((GA, TN, TX)))
display(widgets.HBox((NC, FL, AL)))

In [ ]:
#@title <font color='#E04F39'>After choosing the states, select the schools you're interested in</font>
GA_schools = ['GA Augusta University','GA Clark Atlanta University','GA Columbus State University','GA Emory University','GA Georgia Institute of Technology','GA Georgia State University--Atlanta','GA Kennesaw State University','GA Mercer University','GA University of Georgia--Athens','GA Valdosta State University']
AL_schools = ['AL Auburn University--Auburn','AL Samford University','AL Troy University','AL Tuskegee University','AL The University of Alabama','AL Alabama State University','AL University of North Alabama','AL University of Mobile','AL University of South Alabama','AL University of West Alabama']
FL_schools = ['FL Barry University','FL Ave Maria University','FL Florida Southern College','FL Florida State University','FL Florida Institute of Technology--Melbourne','FL University of Florida--Gainesville','FL University of Central Florida','FL University of Miami--Coral Gables','FL The University of Tampa','FL University of South Florida']
NC_schools = ['NC Appalachian State University','NC Campbell University','NC Duke University','NC East Carolina University','NC Elon University','NC North Carolina State University--Raleigh','NC University of North Carolina at Chapel Hill','NC University of North Carolina at Charlotte','NC Wake Forest University','NC Winston-Salem State University']
TN_schools = ['TN Vanderbilt University','TN University of Tennessee--Knoxville','TN The University of the South','TN University of Memphis','TN Tennessee Technological University','TN Carson-Newman University','TN Middle Tennessee State University','TN Lipscomb University','TN Belmont University','TN East Tennessee State University']
TX_schools = ['TX Rice University','TX Baylor University--Waco','TX Texas A&M University--College Station','TX Texas Christian University','TX Texas Wesleyan University','TX University of Houston-Clear Lake','TX The University of Texas at Austin','TX Tarleton State University','TX Trinity University--San Antonio','TX Lamar University']
GA_select = interactive(user_input, response = widgets.SelectMultiple(options = GA_schools, value = GA_schools, description = 'GA Schools', rows = 11))
AL_select = interactive(user_input, response = widgets.SelectMultiple(options = AL_schools, value = AL_schools, description = 'AL Schools', rows = 11))
FL_select = interactive(user_input, response = widgets.SelectMultiple(options = FL_schools, value = FL_schools, description = 'FL Schools', rows = 11))
NC_select = interactive(user_input, response = widgets.SelectMultiple(options = NC_schools, value = NC_schools, description = 'NC Schools', rows = 11))
TN_select = interactive(user_input, response = widgets.SelectMultiple(options = TN_schools, value = TN_schools, description = 'TN Schools', rows = 11))
TX_select = interactive(user_input, response = widgets.SelectMultiple(options = TX_schools, value = TX_schools, description = 'TX Schools', rows = 11))
states = [GA_select, TN_select, TX_select, NC_select, FL_select, AL_select]
tf_mask = [GA.result, TN.result, TX.result, NC.result, FL.result, AL.result]
to_display = [states[i] for i in range(len(states)) if tf_mask[i] == True]

if len(to_display) < 5:
    display(widgets.HBox(tuple(to_display)))
else:
    left_box = HBox(to_display[3:])
    right_box = HBox(to_display[:3])
    display(VBox([left_box, right_box]))

### <font color='#FFCD00'>***You have a chance to add your school (or another school of interest) to do so:***</font>


1.   Go to: https://collegescorecard.ed.gov/
2.   Look up the target college and find the following information:
  *   2 Digit State Code (example: IL)
  *   School name as it appears on the website (example: University of Illinois Urbana-Champaign)
  *   School city (example: Champaign)
3.   Have this data ready to enter in the proceeding cell


> Format to enter: state, school name, city










In [ ]:
#@title <font color='#E04F39'>This allows you to add a specific school</font>
specific_school = input("Did you not see your school and want to attempt to add it? Y/N: ")
if specific_school.strip()[0].lower() == 'y':
    school_detail = input("Please enter the information for the target college (example: GA, Georgia Institute of Technology, Atlanta): ")
    state, school, city = school_detail.split(',')
    encode = f'{state.strip()} {school.strip()}--{city.strip()}'
    print('Your target school has been added to the data collection')
else:
    encode = 'N'
    print('No school added, please continue')

## <font color='#FFCD00'>***Running the code below will create your data set! Please try to only run the cell below once to avoid overloading the API***</font>

In [ ]:
#@title <font color='#E04F39'>Create your DataFrame (only run once please)</font>

temp_list = [i.result for i in states if type(i.result)!= None]
school_list = set()
for i in range(len(temp_list)):
    if temp_list[i] != None:
        for j in temp_list[i]:
            school_list.add(j)
    else:
        continue
if encode != 'N' and encode not in school_list:
    school_list.add(encode)

def string_convert(college):
    college = college.replace('&', ' ')
    state = college[:2]
    name = college[3:].replace(' ', '%20')
    try:
        clean_name, city = name.split('--')
        return f"latest.school.name={clean_name.replace('--','')}&latest.school.state={state}&latest.school.city={city}&latest.school.operating=1&latest.school.main_campus=1"
    except:
        return f"latest.school.name={name}&latest.school.state={state}&latest.school.operating=1&latest.school.main_campus=1"

def locale(code):
    locale_dict = {11: 'City-Large', 12: 'City-Midsize', 13: 'City-Small', 21: 'Suburb-Large', 22:	'Suburb-Midsize', 23:	'Suburb-Small', 31: 'Town-Fringe', 32: 'Town-Distant', 33: 'Town-Remote', 41: 'Rural-Fringe', 42: 'Rural-Distant', 43: 'Rural-Remote'}
    return locale_dict[code]

def ownership(code):
    ownership_dict = {1: 'Public', 2: 'Private', 3: 'For profit'}
    return ownership_dict[code]

def get_data(school, api_key):
    r = requests.get(f"https://api.data.gov/ed/collegescorecard/v1/schools.json?{string_convert(school)}&fields=latest.school.name,latest.student.demographics.first_generation,latest.student.demographics.median_hh_income,latest.student.retention_rate.overall.full_time,latest.earnings.1_yr_after_completion.median,latest.completion.consumer_rate,latest.cost.avg_net_price.overall,latest.student.size,latest.admissions.admission_rate.overall,latest.student.demographics.race_ethnicity.black,latest.student.demographics.student_faculty_ratio,latest.aid.median_debt_suppressed.completers.overall,latest.school.ownership,latest.school.locale&api_key={api_key}")
    data = r.json()
    parse = data['results'][0]
    college_data = {}
    college_data['College Name'] = parse['latest.school.name']
    college_data['PCT First Gen'] = parse['latest.student.demographics.first_generation']
    college_data['Median hh Income'] = parse['latest.student.demographics.median_hh_income']
    college_data['Retention Rate'] = parse['latest.student.retention_rate.overall.full_time']
    college_data['Median Salary 1yr'] = parse['latest.earnings.1_yr_after_completion.median']
    college_data['Completion Rate'] = parse['latest.completion.consumer_rate']
    college_data['Net Price'] = parse['latest.cost.avg_net_price.overall']
    college_data['Student Size'] = parse['latest.student.size']
    college_data['Admission Rate'] = parse['latest.admissions.admission_rate.overall']
    college_data['PCT Black Students'] = parse['latest.student.demographics.race_ethnicity.black']
    college_data['Student Faculty Ratio'] = parse['latest.student.demographics.student_faculty_ratio']
    college_data['Median Graduation Debt'] = parse['latest.aid.median_debt_suppressed.completers.overall']
    college_data['Control'] = ownership(parse['latest.school.ownership'])
    college_data['Locale'] = locale(parse['latest.school.locale'])
    df = pd.DataFrame([college_data])
    return df

if api_key != 'N':
    dataframe_list = []
    school_list = list(school_list)
    for i in range(len(school_list)):
        data = get_data(school_list[i], api_key)
        dataframe_list.append(data)
    df = pd.concat(dataframe_list, ignore_index = True, sort = False)
    print('Your data has successfully compiled!')
else:
    try:
        df = pd.read_csv('SAIR_Colleges.csv')
    except:
        df = pd.read_csv('https://raw.githubusercontent.com/dlyczak3/SAIR_2023/main/SAIR_Colleges.csv?token=GHSAT0AAAAAACIU36VSFIIXLP6YOPGRCVBKZJD7Y5Q')
    finally:
        print('Your data has is ready to be used!')

print('Here is a sample of your data:')
df.head()

## <font color='#2961FF'>**Stage 2: Examples of Interactive Data Exploration**</font>

In [ ]:
#@title  <font color='#E04F39'>Let's find ways to sort your DataFrame</font>
sort_by = interactive(user_input, response = widgets.ToggleButtons(options = [i for i in df.columns if df[i].dtype != 'object'], description = 'Sort By:', disabled = False, button_style = ''));
asce = interactive(user_input, response = widgets.Checkbox(value = False, description = 'Check for Ascending', disabled = False, indent = False,));

display(widgets.HBox((asce, sort_by)))

In [ ]:
#@title <font color='#E04F39'>Run this cell to display your sorted data</font>
sorted_df = df.sort_values(by = sort_by.result, ascending = asce.result)
sorted_df = sorted_df[["College Name", sort_by.result]]
sorted_df.head(10)

In [ ]:
#@title <font color='#E04F39'>Let's set ranges to find a group of schools who match certain criteria</font>
column1 = 'Student Size'
column2 = 'Admission Rate'
column3 = 'Median Salary 1yr'
column4 = 'Completion Rate'

population = interactive(user_input, response = widgets.IntRangeSlider(min=min(df[column1])-100,max=max(df[column1])+100,step=1,description='Student Size:',disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='d',layout=Layout(width='400px')));
admit_rate = interactive(user_input, response = widgets.FloatRangeSlider(min=max(min(df[column2])-0.01,0),max=min(max(df[column2])+0.01,1.0),step=0.001,description='Admit Rate:',disabled=False,continuous_update=False,orientation='horizontal',readout=True, readout_format='.3f',layout=Layout(width='400px')));
grad_salary = interactive(user_input, response = widgets.IntRangeSlider(min=min(df[column3])-1000,max=max(df[column3])+1000,step=1,description='Grad Salary:',disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='d',layout=Layout(width='400px')));
completion = interactive(user_input, response = widgets.FloatRangeSlider(min=max(min(df[column4])-0.05,0),max=min(max(df[column4])+0.05,1.0),step=0.001,description='Completion Rate:',disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='.3f',layout=Layout(width='400px'),style={'description_width': 'initial'}));

left_box = VBox([population, admit_rate])
right_box = VBox([grad_salary, completion])
HBox([left_box, right_box])

In [ ]:
#@title <font color='#E04F39'>Run this cell to view school(s) matching your criteria</font>

filtered_df = df[(df[column1].between(population.result[0], population.result[1]) & df[column2].between(admit_rate.result[0], admit_rate.result[1]) & df[column3].between(grad_salary.result[0], grad_salary.result[1]) & df[column4].between(completion.result[0], completion.result[1]))]

display(filtered_df)

## <font color='#2961FF'>**Stage 3: let's visualize our data!**</font>

<font color='#64CCC9'>We will be making a parallel coordinates graph. For more information on them: https://www.data-to-viz.com/graph/parallel.html</font>

<font color='#64CCC9'>To generate your graph, run the cell below and select the layers you want to visualize</font>

In [ ]:
#@title  <font color='#E04F39'>Let's set the order and features for your parallel coordinates</font>

num_cats = [i for i in df.columns if df[i].dtype != 'object'][:-1]
variable_one = interactive(user_input, response = widgets.RadioButtons(options=num_cats, layout = {'width': 'max-content'}, value = 'Admission Rate', description = 'Layer One:', disabled=False));
variable_two = interactive(user_input, response = widgets.RadioButtons(options=num_cats, layout = {'width': 'max-content'}, value = 'Student Size', description = 'Layer Two:', disabled=False));
variable_three = interactive(user_input, response = widgets.RadioButtons(options=num_cats, layout = {'width': 'max-content'}, value = 'Completion Rate', description = 'Layer Three:', disabled=False));
variable_four = interactive(user_input, response = widgets.RadioButtons(options=num_cats, layout = {'width': 'max-content'}, value = 'Median Salary 1yr', description = 'Layer Four:', disabled=False));

display(widgets.HBox((variable_one, variable_two,variable_three,variable_four )))

In [ ]:
#@title  <font color='#E04F39'>Run this cell to generate your graph!</font>

graph_df = df.sort_values(by = ['College Name'], ascending = True)
graph_df['idx'] = graph_df.index

n_colors = len(graph_df)
colors = px.colors.sample_colorscale("Portland", [n/(n_colors - 1) for n in range(n_colors)])

fig = go.Figure(data=
    go.Parcoords(
        line = dict(color = graph_df['idx'],
                   colorscale = colors,
                   showscale = False),
        dimensions = list([
            dict(range=[0,max(graph_df['idx'])],
                       tickvals = graph_df['idx'], ticktext = graph_df['College Name'],
                       label ='College Name', values = graph_df['idx']),
            dict(range = [min(graph_df[variable_one.result]),max(graph_df[variable_one.result])],
                 label = variable_one.result, values = graph_df[variable_one.result]),
            dict(range = [min(graph_df[variable_two.result]),max(graph_df[variable_two.result])],
                 label = variable_two.result, values = graph_df[variable_two.result]),
            dict(range = [min(graph_df[variable_three.result]),max(graph_df[variable_three.result])],
                 label = variable_three.result, values = graph_df[variable_three.result]),
            dict(range = [min(graph_df[variable_four.result]),max(graph_df[variable_four.result])],
                 label = variable_four.result, values = graph_df[variable_four.result])])))
fig.update_layout(width = 1700, height = max(n_colors*20, 800), plot_bgcolor ='#54585A', autosize = False, margin = dict(l = 400, t = 60, b = 50), font_size = 18, font_color = '#54585A')
fig.show()

## <font color='#64CCC9'>General note on when to use a table vs graph</font>
### Aurthor Stephen Few's principles from Show Me the Numbers</font>

*   Use Tables when:
     1.   The document will be used to lookup individual values
     2.   The document will compare individual values
     3.   Precise values are required
     4.   The Quantitative info involves more than one unit of measure


*   Use Graphs when:
     1.   The message is contained in the shape of the values
     2.   The document will be used to reveal value relationships

###### <font color='#64CCC9'>ref: https://courses.washington.edu/info424/2007/readings/Show_Me_the_Numbers_v2.pdf</font>

## <font color='#2961FF'>**Stage 4: Free Range**</font>

<font color='#FFCD00'>You now have a chance to play around with the data! If you're not familiar, the library we are primarily using is called "Pandas"</font>

<font color='#FFCD00'>Some simple hints are included, please feel free to use them or explore on your own! Google is your friend</font>

In [ ]:
#@title  <font color='#E04F39'>Setup your sandbox (rerunning this cell restarts your DataFrame)</font>
data = df.copy()
print('Your DataFrame is called \'Data\'')
print('Below are your DataFrame\'s columns:')
for i in data.columns:
    print(i)

In [ ]:
#@title  <font color='#E04F39'>Generate some data exploration ideas</font>

def ideas(name):
    if name == 'Setup':
        return widgets.HTML(value = "These tabs contain ideas for analysis and to help guide you if you need some help or inspiration. <br> Please feel free to add your own cells below (Code or Markdown) <br> Use the information above to access and explore your data <br> If you're comfortable and want to do your own analysis, that's okay too! ")
    if name == 'Describe Data':
        return widgets.HTML(value = "Copy and paste the following into a new Code tab and run it: <font color='#E04F39'>data.describe()</font> <br> What does this information tell you? <br> What questions does it make you ask, and how could you go about answering them? <br> Create a new Markdown slide and writedown your thoughts ")
    if name == 'Correlation':
        return widgets.HTML(value = "Let's start by making a binary variable (either 1 for yes or 0 for no) for the 'Control' feature: <font color='#E04F39'>data['Public School'] = data.Control.apply(lambda x: 1 if x == \"Public\" else 0)</font> <br> Next, let's make a quick correlation matrix with this code: <font color='#E04F39'>data.corr(numeric_only = True)</font> <br> What do you see with from this chart? What would be your next step(s)? <br> Here is a ref: <a href=\"https://www.w3schools.com/python/pandas/pandas_correlations.asp\" target=\"_blank\">Guide to Pandas correlation</a> ")
    if name == 'Custom Criteria':
        return widgets.HTML(value = "You can access a column / feature in your data with the following format ==> data[\'column name\'] <br> Build some custom criteria to find schools who match your conditions <br> For help, ref: <a href=\"https://www.geeksforgeeks.org/filter-pandas-dataframe-with-multiple-conditions/\" target=\"_blank\"> Quick guide to Pandas filtering</a>")
    if name == 'Simple Graphs':
        return widgets.HTML(value = "Why not explore some simple visualizations! <br> Your data is stored as a Pandas df, so it is easy to make simple charts <br> If you're interested, check out this ref: <a href=\"https://www.w3schools.com/python/pandas/pandas_plotting.asp\" target=\"_blank\">Quick Pandas Plotting</a>")
    else:
        return widgets.HTML(value = "taco")
tab_contents = ['Setup', 'Describe Data', 'Correlation', 'Custom Criteria', 'Simple Graphs']
children = [ideas(name) for name in tab_contents]
tab = widgets.Tab()
tab.children = children
tab.titles = [str(i) for i in range(len(children))]
for i in range (len(tab_contents)):
    tab.set_title(i,"{}".format(tab_contents[i]))
tab